In [ ]:
import os
from functools import partial

import matplotlib
matplotlib.use('Agg')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skopt import gbrt_minimize
from pandas import DataFrame as DF

import mcbn.data.dataset_loaders as dl
from mcbn.data.dataset import Dataset

from mcbn.models.model_bn import ModelBN
from mcbn.models.model_do import ModelDO

from mcbn.utils.metrics import pll
from mcbn.utils.metrics import crps
from mcbn.utils.helper import get_setup
from mcbn.utils.helper import get_grid_search_results
from mcbn.utils.helper import dump_yaml
from mcbn.utils.helper import get_new_dir_in_parent_path
from mcbn.utils.helper import make_path_if_missing
from mcbn.utils.helper import get_logger
from mcbn.environment.constants import TAU_EVAL_PATH

In [ ]:
logger = get_logger()

logger.info("STEP 4: Running TAU optimization")

RANDOM_SEED_NP = 2
RANDOM_SEED_TF = 1

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
def run(c, X, y, tau):
    tau = tau[0]
    
    avg_metric_vals = []
    
    # Get path of trained models for all folds
    trained_models_path = os.path.join(os.getcwd(), c['trained_models_path'])
    
    # Get path of indices for all folds
    folds_indices_path = os.path.join(trained_models_path, '..', '..', '..', 'fold_indices')
    
    # Iterate over folds
    for fold_index, fold_model_dir in enumerate(sorted(os.listdir(trained_models_path))):
        
        # Get path of trained model for this fold
        fold_model_path = os.path.join(trained_models_path, fold_model_dir)
        
        # Create a dataset from fold indices
        X_train, y_train, X_val, y_val = dl.load_fold(folds_indices_path, fold_index, X, y)
        dataset = Dataset(X_train, 
                          y_train, 
                          X_val, 
                          y_val, 
                          s['discard_leftovers'],
                          normalize_X=s['normalize_X'], 
                          normalize_y=s['normalize_y'])
            
        # Load the saved model
        with tf.Graph().as_default() as g:
            
            with g.device('/cpu:0'):
                
                # Set random generator seed for reproducible models
                tf.set_random_seed(RANDOM_SEED_TF)
                
                with tf.name_scope("model_{}".format(fold_index)) as scope:
                    if c['base_model_name'] in ['BN', 'MCBN']:
                        model = ModelBN(c['n_hidden'],
                                        K=c['k'],
                                        nonlinearity=c['nonlinearity'],
                                        bn=True,
                                        do=False,
                                        tau=tau,
                                        dataset=dataset,
                                        in_dim=c['in_dim'],
                                        out_dim=c['out_dim'])
                    elif c['base_model_name'] in ['DO', 'MCDO']:
                        keep_prob = 1 - c['dropout']
                        model = ModelDO(c['n_hidden'], 
                                        K=c['k'], 
                                        nonlinearity=c['nonlinearity'], 
                                        bn=False, 
                                        do=True,
                                        tau=tau, 
                                        dataset=dataset, 
                                        in_dim=c['in_dim'], 
                                        out_dim=c['out_dim'],
                                        first_layer_do=True)

                    model.initialize(l2_lambda=c['lambda'], learning_rate=c['learning_rate'])

            #Start session (regular session is default session in with statement)
            saver = tf.train.Saver()
            with tf.Session(config=tf.ConfigProto(
                    allow_soft_placement=False,
                    log_device_placement=False,
                    inter_op_parallelism_threads=1,
                    intra_op_parallelism_threads=1)) as sess:
                
                saver.restore(sess, tf.train.latest_checkpoint(fold_model_path))

                #Get CV PLL
                if c['model_name'] == 'MCBN':
                    samples = model.get_mc_samples(c['mc_samples'], dataset.X_test, c['batch_size'])
                    mean, var = model.get_mc_moments(samples)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(samples, dataset.y_test, c['mc_samples'], tau)
                    else:
                        avg_metric = crps(dataset.y_test, mean, var)
                
                elif c['model_name'] == 'MCBN const':
                    samples = model.get_mc_samples(c['mc_samples'], dataset.X_test, c['batch_size'])
                    mean, var = model.get_mc_moments(samples)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(np.array([mean]), dataset.y_test, 1, tau)
                    else:
                        avg_metric = crps(dataset.y_test, mean, tau**(-1))
                    
                elif c['model_name'] == 'BN':
                    model.update_layer_statistics(dataset.X_test)
                    samples = model.predict(dataset.X_test)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(np.array([samples]), dataset.y_test, 1, tau)
                    else:
                        avg_metric = crps(dataset.y_test, samples, tau**(-1))
                    
                    
                elif c['model_name'] == 'MCDO':
                    samples = model.get_mc_samples(c['mc_samples'], dataset.X_test, keep_prob)
                    mean, var = model.get_mc_moments(samples)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(samples, dataset.y_test, c['mc_samples'], tau)
                    else:
                        avg_metric = crps(dataset.y_test, mean, var)
                    
                elif c['model_name'] == 'MCDO const':
                    samples = model.get_mc_samples(c['mc_samples'], dataset.X_test, keep_prob)
                    mean, var = model.get_mc_moments(samples)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(np.array([mean]), dataset.y_test, 1, tau)
                    else:
                        avg_metric = crps(dataset.y_test, mean, tau**(-1))
                
                elif c['model_name'] == 'DO':
                    samples = model.predict(dataset.X_test, 1)
                    if s['tau_opt_metric'] == 'PLL':
                        avg_metric = pll(np.array([samples]), dataset.y_test, 1, tau)
                    else:
                        avg_metric = crps(dataset.y_test, samples, tau**(-1))

        avg_metric_vals += [avg_metric]
    
    ALL_TAUS.append(tau)
    ALL_PLLS.append(np.mean(avg_metric_vals))
    
    logger.info("tau {:.10f}: Avg {} {:.4f}".format(tau, s['tau_opt_metric'], np.mean(avg_metric_vals)))
    
    #Minimization objective: avg CRPS or avg. negative PLLs
    return (-1 if s['tau_opt_metric'] == 'PLL' else 1) * np.mean(avg_metric_vals)

In [ ]:
def save_results(tau_opt, eval_path, c):
    
    # Get dataset dir
    dataset_path = os.path.join(eval_path, c['dataset_name'])
    make_path_if_missing(dataset_path)
    
    #Save csv for model
    df = DF({'tau':ALL_TAUS, 'pll':ALL_PLLS})
    df.to_csv(os.path.join(dataset_path, c['model_name'] + '.csv'))
    
    #Save plot for model
    fig = plt.figure(figsize=(15,10))
    plt.scatter(ALL_TAUS, ALL_PLLS)
    plt.xlabel('TAU')
    plt.ylabel('PLL')
    plt.tight_layout()
    plt.savefig(os.path.join(dataset_path, c['model_name'] + '.png'))
    
    #Save best result for model
    np.savetxt(os.path.join(dataset_path, c['model_name'] + '_opt.txt'), [tau_opt])

In [ ]:
s = get_setup()
g = get_grid_search_results()

# Create parent evaluation dir
eval_path = get_new_dir_in_parent_path(TAU_EVAL_PATH)

# Save used setup and grid search results
dump_yaml(s, eval_path, 'eval_setup.yml')
dump_yaml(g, eval_path, 'eval_grid_search_results.yml')



# Iterate over each dataset
for dataset_name, models_dict in g.iteritems():
    
    logger.info("Dataset: " + dataset_name)
    
    # Load dataset in memory
    X, y = dl.load_uci_data_full(dataset_name)
    feature_indices, target_indices = dl.load_uci_info(dataset_name)
    
    # Get min and max of optimization range for tau
    ranges = s['tau_range'][dataset_name]
    tau_min = ranges[0]
    tau_max = ranges[1]
    
    # Add 'const' variants of MC models if present
    optimized_MC_models = [m for m in models_dict.keys() if 'MC' in m]
    for mc_model in optimized_MC_models:
        models_dict[mc_model + ' const'] = models_dict[mc_model]
        
    # Iterate over each model's optimized learning parameters for this dataset
    for model_name, opt_dict in models_dict.iteritems():
        
        # Set random generator seed for identical batch picks for standard and const variant
        np.random.seed(RANDOM_SEED_NP)
        
        logger.info("Model: " + model_name)
            
        # Get dataset configuration
        c = {'dataset_name': dataset_name,
             'base_model_name': model_name.replace(' const', ''),
             'model_name': model_name,
             'in_dim': len(feature_indices),
             'out_dim': len(target_indices),
             'n_hidden': s['n_hidden'],
             'k': s['k_specific'].get(dataset_name) or s['k'],
             'nonlinearity': s['nonlinearity'],
             'lambda': opt_dict['lambda'],
             'batch_size': opt_dict['batch_size'],
             'trained_models_path': opt_dict['path'],
             'dropout': opt_dict.get('dropout'), # Can be None
             'learning_rate': s['learning_rate'],
             'mc_samples': s['mc_samples']
             }

        # Save incremental results during optimization
        ALL_TAUS = []
        ALL_PLLS = []
        
        # Make optimization run take extra arguments
        optimize_fun = partial(run, c, X, y)      
        
        tau_opt = gbrt_minimize(optimize_fun, 
                                [(tau_min, tau_max)], 
                                n_random_starts=s['tau_opt_n_random_calls'], 
                                n_calls=s['tau_opt_n_total_calls'])
        save_results(tau_opt.x[0], eval_path, c)
        
logger.info("DONE STEP 4")